In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
#initialize dimensions
date_dimension = pd.read_csv("date_dimension.csv")
Patient_dimension = pd.read_csv("Patient_dimension.csv")
PHU_dimension = pd.read_csv("PHU_dimension.csv")
mobility_dimension = pd.read_csv("Mobility_dimension.csv") 
weather_dimension = pd.read_csv("Weather_dimension.csv") 
special_measures_dimension = pd.read_csv("special_measures_dimension.csv")

In [7]:
#initialize positive cases data
positive_cases = pd.read_csv("covid_patients.csv")

#Only get Ottawa and Toronto cases
cities = ["Ottawa", "Toronto"]
positive_cases = positive_cases[positive_cases.Reporting_PHU_City.isin(cities)]


#limit the date between march and june
start_date = '2020-03-01'
end_date = '2020-06-30'
mask = (positive_cases['Case_Reported_Date'] > start_date) & (positive_cases['Case_Reported_Date'] <= end_date)
positive_cases = positive_cases.loc[mask]
positive_cases

,_id,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
225,226,226,2020-04-28T00:00:00,2020-05-03T00:00:00,2020-05-03T00:00:00,2020-04-30T00:00:00,40s,FEMALE,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
255,256,256,2020-03-24T00:00:00,2020-03-25T00:00:00,2020-03-24T00:00:00,2020-03-24T00:00:00,50s,MALE,NO KNOWN EPI LINK,Resolved,NaN,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
312,313,313,2020-04-02T00:00:00,2020-04-06T00:00:00,2020-04-29T00:00:00,2020-04-02T00:00:00,90+,FEMALE,OB,Fatal,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
334,335,335,2020-04-10T00:00:00,2020-04-17T00:00:00,2020-04-11T00:00:00,2020-04-10T00:00:00,80s,FEMALE,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
386,387,387,2020-04-17T00:00:00,2020-04-20T00:00:00,2020-04-20T00:00:00,2020-04-19T00:00:00,30s,FEMALE,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300535,300536,300536,2020-06-08T00:00:00,2020-06-09T00:00:00,2020-06-09T00:00:00,2020-06-08T00:00:00,<20,UNSPECIFIED,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
300558,300559,300559,2020-05-19T00:00:00,2020-05-20T00:00:00,2020-05-20T00:00:00,2020-05-19T00:00:00,90+,FEMALE,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
300559,300560,300560,2020-06-05T00:00:00,2020-06-05T00:00:00,2020-06-05T00:00:00,2020-06-05T00:00:00,20s,MALE,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
300561,300562,300562,2020-04-24T00:00:00,2020-04-27T00:00:00,2020-04-27T00:00:00,2020-04-24T00:00:00,90+,FEMALE,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358


In [8]:
#initialize ID version of fact table
fact_table = pd.DataFrame(columns=['onset_date_ID', 'reported_date_ID', 'test_date_ID', 'specimen_date_ID', 'patient_ID', 'phu_ID', 
                                   'mobility_ID', 'weather_ID', 'special_measures_ID', 'resolved', 'unresolved', 'fatal'])

In [9]:
# Location ID/Station dictionaries
locations_dict = {'Ottawa': 'ChIJmcB2guBn0UwRr0RW_iic0LY', 'Toronto': 'ChIJ5b2RG4_L1IkRDtQ2gFEjLv4'}
stations_dict = {'Ottawa': '6106001', 'Toronto': '6158359'}

In [ ]:
#Stage fact_table ID FKs and measures
for index, row in positive_cases.iterrows():
    #Missing Date values check (if any date is a float, it means they have missing date data and should be skipped)
    if isinstance(row['Accurate_Episode_Date'], str) and isinstance(row['Case_Reported_Date'], str) and isinstance(row['Test_Reported_Date'], str) and isinstance(row['Specimen_Date'], str):
    
        fact_row = [date_dimension[date_dimension['date'] == row['Accurate_Episode_Date'].split("T")[0]]['date_ID'].values[0],
                    date_dimension[date_dimension['date'] == row['Case_Reported_Date'].split("T")[0]]['date_ID'].values[0],
                    date_dimension[date_dimension['date'] == row['Test_Reported_Date'].split("T")[0]]['date_ID'].values[0],
                    date_dimension[date_dimension['date'] == row['Specimen_Date'].split("T")[0]]['date_ID'].values[0],
                    Patient_dimension[Patient_dimension['patient_ID']==row['Row_ID']]['patient_ID'].values[0],
                    PHU_dimension[PHU_dimension['phu_ID']==row['Reporting_PHU_ID']]['phu_ID'].values[0],
               ]
        
        row_reported_date_str = row[3].split('T')[0]
        row_reported_date = datetime.strptime(row_reported_date_str, '%Y-%m-%d')
        
        # Map mobility_ID
        phu_df = pd.read_csv('PHU_dimension.csv')
        
        phu_city = phu_df.loc[phu_df['phu_ID'] == fact_row[5]]['city'].values[0]
        location_id = locations_dict[phu_city]
        mobility_id = location_id + "+" + row_reported_date_str
        fact_row.append(mobility_id)
        
        # Map weather_ID
        station = stations_dict[phu_city]
        weather_id = station + "+" +row_reported_date_str
        fact_row.append(weather_id)
        
        # Map special_measures_ID
        sm_df = pd.read_csv('special_measures_dimension.csv')
        
        special_measures_id = None
        for index, sm_row in sm_df.iterrows():
               
            sm_row_start_date = sm_row['start_date']
            sm_row_end_date = sm_row['end_date']
            
            # If row date within range of special measure make measure_ID FK
            if isinstance(sm_row_end_date, str):
                sm_row_end_date = datetime.strptime(sm_row['end_date'], '%Y-%m-%d')
                sm_row_start_date = datetime.strptime(sm_row['start_date'], '%Y-%m-%d')
                if row_reported_date >= sm_row_start_date and row_reported_date < sm_row_end_date:
                    special_measures_id = sm_row['special_measures_ID']
                    break 
            elif isinstance(sm_row_start_date, str):
                sm_row_start_date = datetime.strptime(sm_row['start_date'], '%Y-%m-%d')
                if row_reported_date >= sm_row_start_date:
                    special_measures_id = sm_row['special_measures_ID']
                    break
            else:
                special_measures_id = sm_row['special_measures_ID']
                
        fact_row.append(special_measures_id)  
    
        #set measures
        if (row['Outcome1'] == 'Resolved'):
            fact_row.append(1)
            fact_row.append(0)
            fact_row.append(0)
        elif (row['Outcome1'] == 'Fatal'):
            fact_row.append(0)
            fact_row.append(0)
            fact_row.append(1)
        else: #If **Not Resolved** or not any other value we assume it is unresolved
            fact_row.append(0)
            fact_row.append(1)
            fact_row.append(0)
        
        fact_table.loc[len(fact_table)] = fact_row

fact_table.head()

In [7]:
positive_cases.to_csv("test_ott_tor.csv",index=False) #can be removed when done
fact_table.to_csv("fact_table_rn.csv",index=False) #can be removed when done

In [8]:
final_fact_table = pd.DataFrame(columns=['onset_date_surrogate', 'reported_date_surrogate', 'test_date_surrogate', 'specimen_date_surrogate', 'patient_surrogate', 'phu_surrogate', 
                                   'mobility_surrogate', 'weather_surrogate', 'special_measures_surrogate', 'resolved', 'unresolved', 'fatal'])

In [9]:
#Replacing with surrogate keys
for idx, row in fact_table.iterrows():
    print(row['mobility_ID'])
    fact_row = [date_dimension[date_dimension['date_ID'] == row['onset_date_ID']]['date_key'].values[0],
                date_dimension[date_dimension['date_ID'] == row['reported_date_ID']]['date_key'].values[0],
                date_dimension[date_dimension['date_ID'] == row['test_date_ID']]['date_key'].values[0],
                date_dimension[date_dimension['date_ID'] == row['specimen_date_ID']]['date_key'].values[0],
                Patient_dimension[Patient_dimension['patient_ID']==row['patient_ID']]['patient_key'].values[0],
                PHU_dimension[PHU_dimension['phu_ID']==row['phu_ID']]['phu_key'].values[0],
                mobility_dimension[mobility_dimension['mobility_ID']==row['mobility_ID']]['mobility_key'].values[0],
                weather_dimension[weather_dimension['weather_ID']==row['weather_ID']]['weather_key'].values[0],
                special_measures_dimension[special_measures_dimension['special_measures_ID']==row['special_measures_ID']]['special_measures_key'].values[0],
                row["resolved"], row["unresolved"],row["fatal"]
               ] 
    final_fact_table.loc[len(final_fact_table)] = fact_row
final_fact_table.head()

ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-05-03
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-03-25
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-04-06
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-04-17
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-04-20
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-04-22
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-04-14
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-05-09
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-05-31
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-03-16
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-03-23
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-03-24
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-03-27
ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-28
ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-29
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-05-06
ChIJmcB2guBn0UwRr0RW_iic0LY+2020-04-17
ChIJmcB2guBn0UwRr0RW_iic0LY+2020-04-07
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-06-11
ChIJmcB2guBn0UwRr0RW_iic0LY+2020-04-02
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-04-28
ChIJmcB2guBn0UwRr0RW_iic0LY+2020-05-22
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-05-15
ChIJmcB2guBn0UwRr0RW_iic0LY+2020-04-01
ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-04-03
ChIJmcB2guBn0UwRr0RW_iic0

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
final_fact_table.to_csv("fact_table_dimension.csv",index=False) #can be removed when done